In [1]:
import os
import numpy as np
import math
import re
import random
import shutil
import gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_addons as tfa
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet as efn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tensorflow.keras.constraints import Constraint
from scipy.spatial.distance import squareform
%matplotlib inline
from toolz import interleave
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.model_selection import KFold,StratifiedKFold

print("Tensorflow version " + tf.__version__)

C:\Users\erfan\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Tensorflow version 2.13.0


In [5]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = strategy.num_replicas_in_sync
# Number of computing cores, is 8 for a TPU V3-8
print(f'N_REPLICAS: {N_REPLICAS}')

Running on GPU
N_REPLICAS: 1


In [3]:
class DataLoader:
    """
    If the reference is unphased, cannot handle phased target data, so the valid (ref, target) combinations are:
    (phased, phased), (phased, unphased), (unphased, unphased)
    Important note: for each case, the model should be trained separately
    """
    def __init__(self, reference_panel_file_path, target_file_path):
        self.ref_n_header_lines = []
        self.ref_n_data_header = ""
        self.map_values_1_vec = np.vectorize(self.map_hap_2_ind_parent_1)
        self.map_values_2_vec = np.vectorize(self.map_hap_2_ind_parent_2)
        print("Rading the reference file...")
        # get header
        root, ext = os.path.splitext(reference_panel_file_path)
        with gzip.open(reference_panel_file_path, 'rt') if ext == '.gz' else open(reference_panel_file_path, 'rt') as f_in:
            # skip info
            while True:
                line = f_in.readline()
                if line.startswith("##"):
                    self.ref_n_header_lines.append(line)
                else:
                    self.ref_n_data_header = line
                    break
        self.reference_panel = pd.read_csv(reference_panel_file_path,
                                           comment='#',
                                           sep='\t',
                                           names=self.ref_n_data_header.strip().split('\t'))
        self.VARIANT_COUNT = self.reference_panel.shape[0]
        print(f"{self.VARIANT_COUNT} variants found. Done!")
        print("Rading the target file...")
        self.target_n_header_lines = []
        self.target_n_data_header = ""
        root, ext = os.path.splitext(target_file_path)
        # get header
        with gzip.open(target_file_path, 'rt') if ext == '.gz' else open(target_file_path, 'rt') as f_in:
            # skip info
            while True:
                line = f_in.readline()
                if line.startswith("##"):
                    self.target_n_header_lines.append(line)
                else:
                    self.target_n_data_header = line
                    break
        real_target_set = pd.read_csv(target_file_path,
                                           comment='#',
                                           sep='\t',
                                           names=self.target_n_data_header.strip().split('\t'),)
        print(f"{real_target_set.shape[0]} variants found. Done!")
        target_is_phased = "|" in real_target_set.iloc[0, 10]
        ref_is_phased = "|" in self.reference_panel.iloc[0, 10]
        self.is_phased = target_is_phased and ref_is_phased
        print("Creating the new target dataframe")
        self.target_set = real_target_set.merge(self.reference_panel["ID"], on='ID', how='right')
        self.target_set[self.reference_panel.columns[:9]] = self.reference_panel[self.reference_panel.columns[:9]]
        self.target_set.fillna(".|." if self.is_phased else "./.", inplace=True)
        print("Extracting genotype information...")
        SEP = "|" if self.is_phased else "/"
        def get_num_allels(g):
            v1, v2 = g.split(SEP)
            return max(int(v1), int(v2)) + 1

        def key_gen(v1, v2):
            return f"{v1}{SEP}{v2}"

        genotype_vals = np.unique(self.reference_panel.iloc[:, 9:].values)
        if target_is_phased != ref_is_phased:
            phased_to_unphased_dict = {}
            for i in range(genotype_vals.shape[0]):
                key = genotype_vals[i]
                v1, v2 = [int(s) for s in genotype_vals[i].split("|")]
                genotype_vals[i] = f"{min(v1, v2)}{SEP}{max(v1, v2)}"
                phased_to_unphased_dict[key] = genotype_vals[i]
            self.reference_panel.replace(phased_to_unphased_dict, inplace=True)
        genotype_vals = np.unique(genotype_vals)
        allele_count = max(map(get_num_allels, genotype_vals))
        if self.is_phased:
            self.hap_map = {str(i): i for i in range(allele_count)}
            self.hap_map.update({".": allele_count})
            self.r_hap_map = {i:k for k, i in self.hap_map.items()}
            self.map_preds_2_allele = np.vectorize(lambda x: self.r_hap_map[x])
        self.MISSING_VALUE = self.SEQ_DEPTH = allele_count + 1 if self.is_phased else len(genotype_vals) + 1
        self.genotype_keys = np.array([key_gen(i,j) for i in range(allele_count) for j in range(allele_count)]) if self.is_phased else genotype_vals
        self.genotype_keys = np.hstack([self.genotype_keys, [".|."] if self.is_phased else ["./."]])
        self.replacement_dict = {g:i for i,g in enumerate(self.genotype_keys)}
        self.reverse_replacement_dict = {i:g for g,i in self.replacement_dict.items()}

    def map_hap_2_ind_parent_1(self, x):
        return self.hap_map[x.split('|')[0]]

    def map_hap_2_ind_parent_2(self, x):
        return self.hap_map[x.split('|')[1]]

    def __get_forward_data(self, data: pd.DataFrame):
        if self.is_phased:
            # break it into haplotypes
            _x = np.empty((data.shape[1] * 2, data.shape[0]), dtype=np.int32)

            _x[0::2] = self.map_values_1_vec(data.values.T)
            _x[1::2] = self.map_values_2_vec(data.values.T)
            return _x
        else:
            return data.replace(self.replacement_dict).values.T.astype(np.int32)

    def get_ref_set(self, starting_var_index=None, ending_var_index=None):
        if starting_var_index>=0 and ending_var_index>=starting_var_index:
            return self.__get_forward_data(self.reference_panel.iloc[starting_var_index:ending_var_index, 9:])
        else:
            print("No variant indices provided or indices not valid, using the whole sequence...")
            return self.__get_forward_data(self.reference_panel.iloc[:, 9:])

    def get_target_set(self, starting_var_index=None, ending_var_index=None):
        if starting_var_index>=0 and ending_var_index>=starting_var_index:
            return self.__get_forward_data(self.target_set.iloc[starting_var_index:ending_var_index, 9:])
        else:
            print("No variant indices provided or indices not valid, using the whole sequence...")
            return self.__get_forward_data(self.target_set.iloc[:, 9:])

    def convert_haps_to_genotypes(self, allele_probs):
      '''output format: GT:DS:GP'''
      FORMAT = "GT:DS:GP"
      n_haploids, n_variants, n_alleles = allele_probs.shape
      allele_probs_normalized = softmax(allele_probs, axis=-1)

      if n_haploids % 2 != 0:
          raise ValueError("Number of haploids should be even.")

      n_samples = n_haploids // 2
      genotypes = np.zeros((n_samples, n_variants), dtype=object)

      for i in tqdm(range(n_samples)):
        haploid_1 = allele_probs_normalized[2 * i]
        haploid_2 = allele_probs_normalized[2 * i + 1]

        for j in range(n_variants):
          phased_probs = np.multiply.outer(haploid_1[j], haploid_2[j]).flatten()
          unphased_probs = np.array([phased_probs[0], sum(phased_probs[1:3]), phased_probs[-1]])
          unphased_probs_str = ",".join([f"{v:.6f}" for v in unphased_probs])
          alt_dosage = np.dot(unphased_probs, [0, 1, 2])
          variant_genotypes = [str(v) for v in np.argmax(allele_probs_normalized[i*2:(i+1)*2, j], axis=-1)]
          genotypes[i, j] = '|'.join(variant_genotypes) + f":{alt_dosage:.3f}:{unphased_probs_str}"

      new_vcf = self.target_set.copy()
      new_vcf.iloc[:n_variants, 9:] = genotypes.T
      new_vcf["FORMAT"] = FORMAT
      new_vcf["QUAL"] = "."
      new_vcf["FILTER"] = "."
      new_vcf["INFO"] = "IMPUTED"
      return new_vcf

    def convert_unphased_probs_to_genotypes(self, allele_probs):
      '''output format: GT:DS:GP'''
      FORMAT = "GT:DS:GP"
      n_samples, n_variants, n_alleles = allele_probs.shape
      allele_probs_normalized = softmax(allele_probs, axis=-1)
      genotypes = np.zeros((n_samples, n_variants), dtype=object)

      for i in tqdm(range(n_samples)):
          for j in range(n_variants):
              unphased_probs = allele_probs_normalized[i, j]
              unphased_probs_str = ",".join([f"{v:.6f}" for v in unphased_probs])
              alt_dosage = np.dot(unphased_probs, [0, 1, 2])
              variant_genotypes = np.vectorize(self.reverse_replacement_dict.get)(np.argmax(unphased_probs, axis=-1)).flatten()
              genotypes[i, j] = '/'.join(variant_genotypes) + f":{unphased_probs_str}:{alt_dosage:.3f}"

      new_vcf = self.target_set.copy()
      new_vcf.iloc[:, 9:] = genotypes.T
      new_vcf["FORMAT"] = FORMAT
      new_vcf["QUAL"] = "."
      new_vcf["FILTER"] = "."
      new_vcf["INFO"] = "IMPUTED"
      return new_vcf

    def __get_headers_for_output(self):
      headers = ["##fileformat=VCFv4.2",
           '''##source=STI v1.0.0''',
           '''##INFO=<ID=IMPUTED,Number=0,Type=Flag,Description="Marker was imputed">''',
           '''##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">''',
           '''##FORMAT=<ID=DS,Number=A,Type=Float,Description="Estimated Alternate Allele Dosage : [P(0/1)+2*P(1/1)]">''',
           '''##FORMAT=<ID=GP,Number=G,Type=Float,Description="Estimated Posterior Probabilities for Genotypes 0/0, 0/1 and 1/1">''']
      return headers

    def preds_to_genotypes(self, preds):
        """
        WARNING: This only supports bi-allelic data right now!
        :param preds: numpy array of (n_samples, n_variants, n_alleles)
        :return: numpy array of the same shape, with genotype calls, e.g., "0/1"
        """
        if self.is_phased:
          return self.convert_haps_to_genotypes(preds)
        else:
          return self.convert_unphased_probs_to_genotypes(preds)

    def write_ligated_results_to_vcf(self, df, file_name):
      with gzip.open(file_name, 'wt') if file_name.endswith(".gz") else open(file_name, 'wt') as f_out:
          # write info
          f_out.write("\n".join(self.__get_headers_for_output())+"\n")
      df.to_csv(file_name, sep="\t", mode='a', index=False)

<function tensorflow.python.eager.context.set_log_device_placement(enabled)>